In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 为了更好地理解代理框架，让我们构建一个具有两个工具的代理: 
# 一个用于在线查找， 另一个用于查找加载到索引中的特定数据。

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

search.invoke('What is the weather in Nanjing')

[{'url': 'https://www.weather-forecast.com/locations/Nanjing/forecasts/latest',
  'content': 'Nanjing Weather  Nanjing Weather Today  Nanjing Weather Next Week  Nanjing Weather Next WeekLive Weather Warnings, hourly weather updates. Accurate Nanjing weather today, forecast for sun, rain, wind and temperature. ... Nanjing Weather Forecast Nanjing Weather Forecast. Nanjing (Jiangsu Sheng, China) forecast issued: 7 am Fri 02 Feb 2024 local time ... Nanjing is 5 m above sea level and located at 32.05° N 118.78° E. Nanjing has a ...'},
 {'url': 'https://www.weather-atlas.com/en/china/nanjing-weather-february',
  'content': 'Average rainfall days in February Nanjing, China  Average daylight in February / Average sunshine in February Nanjing, China  Average rainfall in February Nanjing, China  Average temperature in February Nanjing, ChinaToday Tomorrow Long-term Year J February M A M J J A S O N D °Fahrenheit Expand Contents Weather in February As the city traverses through February, it sees

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

documnents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documnents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [5]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we’re 

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [7]:
tools = [search, retriever_tool]

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0)

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({'input':'hi!'})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing?'}`


You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring​After all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be assigned string tags or key-value metadata, allowing you to attach correlation ids or AB test variants, and filter runs accordingly.We’ve also made it possible to associate feedback programmatically with runs. This means that if your application has a thumbs up/down button on it, you can use that to log feedback back to LangSmith. This can be used to track performance over t

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Editing Examples: You can quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.\n\n2. Monitoring: LangSmith can be used to monitor your application by logging all traces, visualizing latency and token usage statistics, and troubleshooting specific issues as they arise. Each run can also be assigned string tags or key-value metadata, allowing you to attach correlation ids or AB test variants, and filter runs accordingly.\n\n3. Feedback and Performance Tracking: You can associate feedback programmatically with runs, track performance over time, and pinpoint underperforming data points, which can subsequently be added to a dataset for future testing.\n\n4. Exporting Datasets: LangSmith makes it easy to curate datasets, which can be exported for use in other contexts, s

In [20]:
agent_executor.invoke({"input": "whats the weather in NanJing?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Nanjing'}`


SSLError(MaxRetryError("HTTPSConnectionPool(host='api.tavily.com', port=443): Max retries exceeded with url: /search (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))I'm currently unable to retrieve the weather information for Nanjing. I apologize for the inconvenience.

> Finished chain.


{'input': 'whats the weather in NanJing?',
 'output': "I'm currently unable to retrieve the weather information for Nanjing. I apologize for the inconvenience."}

In [21]:
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

Connection error caused failure to patch https://api.smith.langchain.com/runs/c8de35f4-de7d-4ade-8ed9-5775aba6670d  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/c8de35f4-de7d-4ade-8ed9-5775aba6670d (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))
Connection error caused failure to post https://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))"))


In [24]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you further, Bob?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob. How can I assist you further, Bob?'}

: 